In [1]:
import os
import getpass
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
import chromadb

PERSISTENT_DIR = "./chroma_db"
COLLECTION_NAME="rag_mcp"

def init_chroma():
    client = chromadb.PersistentClient(path=PERSISTENT_DIR)

    collection = client.get_or_create_collection(name=COLLECTION_NAME)
    return collection

init_chroma()

def get_chroma_client():
    return chromadb.PersistentClient(path=PERSISTENT_DIR)
